In [1]:
import os
import glob
import itertools

### download the bams

In [2]:
!wget http://popgen.dk/software/download/angsd/bams.tar.gz

--2019-05-21 15:52:09--  http://popgen.dk/software/download/angsd/bams.tar.gz
Resolving popgen.dk (popgen.dk)... 130.225.98.44
Connecting to popgen.dk (popgen.dk)|130.225.98.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102283097 (98M) [application/x-gzip]
Saving to: ‘bams.tar.gz’

bams.tar.gz         100%[===================>]  97.54M   109MB/s    in 0.9s    

2019-05-21 15:52:10 (109 MB/s) - ‘bams.tar.gz’ saved [102283097/102283097]



### untar and index the bams

In [3]:
!tar xf bams.tar.gz
!for i in bams/*.bam;do samtools index $i;done

### download and index the ancestral state fasta file

In [4]:
!wget http://popgen.dk/software/download/angsd/hg19ancNoChr.fa.gz
!samtools faidx hg19ancNoChr.fa.gz

--2019-05-21 15:52:14--  http://popgen.dk/software/download/angsd/hg19ancNoChr.fa.gz
Resolving popgen.dk (popgen.dk)... 130.225.98.44
Connecting to popgen.dk (popgen.dk)|130.225.98.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 850829230 (811M) [application/x-gzip]
Saving to: ‘hg19ancNoChr.fa.gz’

hg19ancNoChr.fa.gz  100%[===================>] 811.41M   112MB/s    in 7.3s    

2019-05-21 15:52:22 (111 MB/s) - ‘hg19ancNoChr.fa.gz’ saved [850829230/850829230]



### make the bam filelists for use by angsd

In [5]:
## make a single overall filelist
!ls bams/*.bam > all.filelist

In [6]:
# one filelist for each sample
samples = []
for sample_bam in glob.glob('bams/*.bam'):
    sampleID = sample_bam.split('/')[1].split('.')[0]
    samples.append(sampleID)
    !echo {sample_bam} > {sampleID}.filelist.ind

### paths to the analysis programs

In [22]:
# will need to be replaced your local installation
ANGSD = '/home/ryan/programs/angsd/angsd'
realSFS = '/home/ryan/programs/angsd/misc/realSFS'
IBS = '/home/ryan/programs/angsd/misc/ibs'

#  realSFS method

In [8]:
!mkdir results_realsfs

In [9]:
for sample in samples:
    !{ANGSD} -b {sample}.filelist.ind \
    -anc hg19ancNoChr.fa.gz \
    -minMapQ 30 -minQ 20 -GL 2 \
    -doSaf 1 -doDepth 1 -doCounts 1 \
    -out {sample}

	-> angsd version: 0.919-43-g0b5946e (htslib: 1.6-2-g8bbc018) build(Oct  6 2017 11:20:17)
	-> Reading fasta: hg19ancNoChr.fa.gz
	-> Parsing 1 number of samples 
	-> Printing at chr: 20 pos:14048474 chunknumber 1800 contains 681 sites
	-> Done reading data waiting for calculations to finish
	-> Done waiting for threads
	-> Output filenames:
		->"smallNA07000.arg"
		->"smallNA07000.saf.gz"
		->"smallNA07000.saf.pos.gz"
		->"smallNA07000.saf.idx"
		->"smallNA07000.depthSample"
		->"smallNA07000.depthGlobal"
	-> Tue May 21 15:53:16 2019
	-> Arguments and parameters for all analysis are located in .arg file
	[ALL done] cpu-time used =  23.97 sec
	[ALL done] walltime used =  24.00 sec
	-> angsd version: 0.919-43-g0b5946e (htslib: 1.6-2-g8bbc018) build(Oct  6 2017 11:20:17)
	-> Reading fasta: hg19ancNoChr.fa.gz
	-> Parsing 1 number of samples 
	-> Printing at chr: 20 pos:14044661 chunknumber 800 contains 1793 sites
	-> Done reading data waiting for calculations to finish
	-> Done waiting for 

In [10]:
# iterate over each pair of samples
pairs = list(itertools.combinations(samples, 2))

In [11]:
# run realSFS for each pair of samples
for sample1, sample2 in pairs:
     !{realSFS} {sample1}.saf.idx {sample2}.saf.idx \
     > ./results_realsfs/{sample1}_{sample2}.2dsfs

	-> Version of fname:smallNA07000.saf.idx is:2
	-> Assuming .saf.gz file: smallNA07000.saf.gz
	-> Assuming .saf.pos.gz: smallNA07000.saf.pos.gz
	-> Version of fname:smallNA11840.saf.idx is:2
	-> Assuming .saf.gz file: smallNA11840.saf.gz
	-> Assuming .saf.pos.gz: smallNA11840.saf.pos.gz
	-> args: tole:0.000001 nthreads:4 maxiter:100 nsites:0 start:(null) chr:(null) start:-1 stop:-1 fstout:(null) oldout:0 seed:1558447011 bootstrap:0 whichFst:0 fold:0 ref:(null) anc:(null)
	-> Multi SFS is 'still' under development. Please report strange behaviour
	-> nSites: 1451016
	-> The choice of -nSites will require atleast: 55.351868 megabyte memory, that is at least: 0.01% of total memory
	-> dim(smallNA07000.saf.idx):3
	-> dim(smallNA11840.saf.idx):3
	-> Dimension of parameter space: 9
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:1
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find interse

	-> Sites to keep[8] from pop0:	61147
	-> Sites to keep[8] from pop1:	61147
	-> [readdata] lastread:61147 posi:14000296
	-> Comparing positions: 1 with 0 has:1046163
	-> Only read nSites: 1046163 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:9
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[9] from pop0:	73554
	-> Sites to keep[9] from pop1:	73554
	-> [readdata] lastread:73554 posi:14000296
	-> Comparing positions: 1 with 0 has:1119717
	-> Only read nSites: 1119717 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Will run optimization on nSites: 1119717
------------
startlik=-2593701.166205
lik[2]=-198864.341967 diff=2.394837e+06 alpha:1.00000

	-> Sites to keep[5] from pop0:	75862
	-> Sites to keep[5] from pop1:	75862
	-> [readdata] lastread:75862 posi:14000253
	-> Comparing positions: 1 with 0 has:904026
	-> Only read nSites: 904026 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:6
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[6] from pop0:	77910
	-> Sites to keep[6] from pop1:	77910
	-> [readdata] lastread:77910 posi:14000253
	-> Comparing positions: 1 with 0 has:981936
	-> Only read nSites: 981936 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:7
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find i

	-> Sites to keep[19] from pop0:	50453
	-> Sites to keep[19] from pop1:	50453
	-> [readdata] lastread:50453 posi:14000319
	-> Comparing positions: 1 with 0 has:469938
	-> Only read nSites: 469938 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:2
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[2] from pop0:	49280
	-> Sites to keep[2] from pop1:	49280
	-> [readdata] lastread:49280 posi:14000319
	-> Comparing positions: 1 with 0 has:519218
	-> Only read nSites: 519218 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:20
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to fin

	-> [readdata] lastread:74980 posi:14000242
	-> Comparing positions: 1 with 0 has:219758
	-> Only read nSites: 219758 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:12
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[12] from pop0:	77814
	-> Sites to keep[12] from pop1:	77814
	-> [readdata] lastread:77814 posi:14000242
	-> Comparing positions: 1 with 0 has:297572
	-> Only read nSites: 297572 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:16
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop

...

	-> [readdata] lastread:90957 posi:14000031
	-> Comparing positions: 1 with 0 has:460525
	-> Only read nSites: 460525 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:17
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[17] from pop0:	81594
	-> Sites to keep[17] from pop1:	81594
	-> [readdata] lastread:81594 posi:14000031
	-> Comparing positions: 1 with 0 has:542119
	-> Only read nSites: 542119 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:18
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop

	-> dim(smallNA11829.saf.idx):3
	-> dim(smallNA07056.saf.idx):3
	-> Dimension of parameter space: 9
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:1
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[1] from pop0:	86585
	-> Sites to keep[1] from pop1:	86585
	-> [readdata] lastread:86585 posi:14000031
	-> Comparing positions: 1 with 0 has:86585
	-> Only read nSites: 86585 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:10
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[10] from pop0:	88530
	-> S

	-> [readdata] lastread:85182 posi:14000031
	-> Comparing positions: 1 with 0 has:1274917
	-> Only read nSites: 1274917 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:8
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[8] from pop0:	84959
	-> Sites to keep[8] from pop1:	84959
	-> [readdata] lastread:84959 posi:14000031
	-> Comparing positions: 1 with 0 has:1359876
	-> Only read nSites: 1359876 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:9
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop

	-> Sites to keep[4] from pop0:	87879
	-> Sites to keep[4] from pop1:	87879
	-> [readdata] lastread:87879 posi:14000031
	-> Comparing positions: 1 with 0 has:1040122
	-> Only read nSites: 1040122 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:5
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[5] from pop0:	91337
	-> Sites to keep[5] from pop1:	91337
	-> [readdata] lastread:91337 posi:14000031
	-> Comparing positions: 1 with 0 has:1131459
	-> Only read nSites: 1131459 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:6
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to fi

	-> [readdata] lastread:84759 posi:14000031
	-> Comparing positions: 1 with 0 has:704110
	-> Only read nSites: 704110 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:2
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, and possibly using -sites
	-> Sites to keep[2] from pop0:	81976
	-> Sites to keep[2] from pop1:	81976
	-> [readdata] lastread:81976 posi:14000031
	-> Comparing positions: 1 with 0 has:786086
	-> Only read nSites: 786086 will therefore prepare next chromosome (or exit)
	-> Done reading data from chromosome will prepare next chromosome
	-> Is in multi sfs, will now read data from chr:20
	-> hello Im the master merge part of realSFS. and I'll now do a tripple bypass to find intersect 
	-> 1) Will set iter according to chooseChr and start and stop, a

### get the R script to analyze the realSFS output

In [12]:
!wget https://raw.githubusercontent.com/rwaples/freqfree_suppl/master/read_realSFS.R

--2019-05-21 15:58:28--  https://raw.githubusercontent.com/rwaples/freqfree_suppl/master/read_realSFS.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1212 (1.2K) [text/plain]
Saving to: ‘read_realSFS.R’

read_realSFS.R      100%[===================>]   1.18K  --.-KB/s    in 0s      

2019-05-21 15:58:29 (81.7 MB/s) - ‘read_realSFS.R’ saved [1212/1212]



### lets take a look at the results for a single pair of individuals

In [13]:
!Rscript \
-e "source('./read_realSFS.R')" \
-e "res = read_realSFS('results_realsfs/smallNA06985_smallNA11830.2dsfs')" \
-e "res['sample1'] = 'smallNA06985'; res['sample2'] = 'smallNA11830'" \
-e "print(res[,c('sample1', 'sample2', 'nSites', 'Kin', 'R0', 'R1') ])"

       sample1      sample2  nSites         Kin        R0        R1
1 smallNA06985 smallNA11830 1372848 -0.04746918 0.6052391 0.3290302


# IBS method

In [16]:
!mkdir results_IBS

### make a genotype likelihood file (glf) file, this file contains all individuals

In [17]:
!{ANGSD} -b all.filelist \
-minMapQ 30 -minQ 20 -GL 2 \
-doGlf 1 \
-out glf

	-> angsd version: 0.919-43-g0b5946e (htslib: 1.6-2-g8bbc018) build(Oct  6 2017 11:20:17)
	-> Parsing 10 number of samples 
	-> Printing at chr: 20 pos:14056689 chunknumber 2700 contains 663 sitess
	-> Done reading data waiting for calculations to finish
	-> Done waiting for threads
	-> Output filenames:
		->"glf.arg"
		->"glf.glf.gz"
	-> Tue May 21 15:58:56 2019
	-> Arguments and parameters for all analysis are located in .arg file
	[ALL done] cpu-time used =  25.55 sec
	[ALL done] walltime used =  27.00 sec


### run the ibs inference 

In [18]:
!{IBS} -glf glf.glf.gz \
-model 0 \
-nInd 10 -allpairs 1 \
-outFileName results_IBS/ibs.model0.results

using seed 0
	-> Dumping file: results_IBS/ibs.model0.results.log
	-> Dumping file: results_IBS/ibs.model0.results.ibspair
reading data
read 1689799 sites
	using model 0 
analysing pair 0 1
	using nSites= 1199765 
	tol reached in 152 interations	diff=0.000000	model=0
analysing pair 0 2
	using nSites= 1127797 
	tol reached in 130 interations	diff=0.000000	model=0
analysing pair 0 3
	using nSites= 1359492 
	tol reached in 49 interations	diff=0.000000	model=0
analysing pair 0 4
	using nSites= 1278230 
	tol reached in 73 interations	diff=0.000000	model=0
analysing pair 0 5
	using nSites= 1355014 
	tol reached in 83 interations	diff=0.000000	model=0
analysing pair 0 6
	using nSites= 1395178 
	tol reached in 59 interations	diff=0.000000	model=0
analysing pair 0 7
	using nSites= 1367560 
	tol reached in 72 interations	diff=0.000000	model=0
analysing pair 0 8
	using nSites= 1066307 
	tol reached in 204 interations	diff=0.000000	model=0
analysing pair 0 9
	using nSites= 1272263 
	tol reached in

In [24]:
!wget https://raw.githubusercontent.com/rwaples/freqfree_suppl/master/read_IBS.R

--2019-05-21 16:23:27--  https://raw.githubusercontent.com/rwaples/freqfree_suppl/master/read_IBS.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4394 (4.3K) [text/plain]
Saving to: ‘read_IBS.R’

read_IBS.R          100%[===================>]   4.29K  --.-KB/s    in 0s      

2019-05-21 16:23:27 (32.4 MB/s) - ‘read_IBS.R’ saved [4394/4394]



In [25]:
!Rscript \
-e "source('./read_IBS.R')" \
-e "res = do_derived_stats(read_ibspair_model0('results_IBS/ibs.model0.results.ibspair'))" \
-e "print(res[6,c('ind1', 'ind2', 'nSites', 'Kin', 'R0', 'R1') ])"

  ind1 ind2  nSites         Kin        R0        R1
6    0    6 1395178 -0.03655172 0.5791045 0.3439425


In [26]:
# the ibs method in angsd reports the individuals as they appear in the filelist
!cat all.filelist

bams/smallNA06985.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA06994.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA07000.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA07056.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA07357.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA11829.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA11830.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA11831.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA11832.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam
bams/smallNA11840.mapped.ILLUMINA.bwa.CEU.low_coverage.20111114.bam


In [28]:
# and compare to the realSFS results
!Rscript \
-e "source('./read_realSFS.R')" \
-e "res = read_realSFS('results_realsfs/smallNA06985_smallNA11830.2dsfs')" \
-e "res['sample1'] = 'smallNA06985'; res['sample2'] = 'smallNA11830'" \
-e "print(res[,c('sample1', 'sample2', 'nSites', 'Kin', 'R0', 'R1') ])"

       sample1      sample2  nSites         Kin        R0        R1
1 smallNA06985 smallNA11830 1372848 -0.04746918 0.6052391 0.3290302
